In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D, LSTM
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
# 文件路径列表
file_paths = [
    'C:\\Users\\mjy\\Desktop\\Project_MSCM\\INPUT_3.csv',
    'C:\\Users\\mjy\\Desktop\\Project_MSCM\\INPUT_7.csv',
    'C:\\Users\\mjy\\Desktop\\Project_MSCM\\INPUT_15.csv',
    'C:\\Users\\mjy\\Desktop\\Project_MSCM\\INPUT_30.csv'
]

In [3]:
# 标签列名
label_columns = ['未来7日涨幅']

# 存储每个标签的准确率
accuracies = {label: [] for label in label_columns}


In [4]:
# 定义 Transformer + LSTM 模型
def create_lstm_transformer_model(input_shape, num_heads=2, ff_dim=64, dropout_rate=0.2, num_transformer_blocks=2):
    inputs = Input(shape=input_shape)

    # LSTM 层提取序列特征
    x = LSTM(64, return_sequences=True)(inputs)

    # 多层 Transformer Encoder
    for _ in range(num_transformer_blocks):
        # Multi-head self-attention
        attention_output = MultiHeadAttention(num_heads=num_heads, key_dim=x.shape[-1])(x, x)
        attention_output = Dropout(dropout_rate)(attention_output)
        attention_output = LayerNormalization(epsilon=1e-6)(attention_output + x)

        # Feed-forward network
        ff_output = Dense(ff_dim, activation='relu')(attention_output)
        ff_output = Dropout(dropout_rate)(ff_output)
        ff_output = Dense(x.shape[-1])(ff_output)
        ff_output = Dropout(dropout_rate)(ff_output)
        x = LayerNormalization(epsilon=1e-6)(ff_output + attention_output)

    # 全局平均池化
    x = GlobalAveragePooling1D()(x)
    x = Dropout(dropout_rate)(x)

    # 输出层
    outputs = Dense(1, activation='sigmoid')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [5]:
# 遍历每个数据集
for file_path in file_paths:
    print(f"Processing file: {file_path}")

    # 读取数据
    data = pd.read_csv(file_path)
    data = data.dropna()

    # 特征和标签
    features = data.iloc[:, 1:192]  # n1-n47, p1-p146
    labels = {label: (data[label] > 0).astype(int) for label in label_columns}

    # 标准化
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    # Reshape data for Transformer input
    features_scaled_reshaped = features_scaled.reshape((features_scaled.shape[0], features_scaled.shape[1], 1))

    # 定义回调函数
    early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)

    # 对每个标签进行训练和评估
    for label_name, label_data in labels.items():
        print(f"Training for label: {label_name}")

        X_train, X_test, y_train, y_test = train_test_split(features_scaled_reshaped, label_data, test_size=0.2, random_state=42)

        # 创建 LSTM + Transformer 模型
        model = create_lstm_transformer_model((features_scaled.shape[1], 1))

        # 训练模型
        model.fit(X_train, y_train, epochs=100, batch_size=16, validation_split=0.2, callbacks=[early_stopping, reduce_lr], verbose=1)

        # 预测并评估
        y_pred = (model.predict(X_test) > 0.5).astype(int)
        accuracy = accuracy_score(y_test, y_pred)
        accuracies[label_name].append(accuracy)

Processing file: C:\Users\mjy\Desktop\Project_MSCM\INPUT_3.csv
Training for label: 未来7日涨幅

Epoch 1/100


34/34 [==============================] - 10s 103ms/step - loss: 0.7532 - accuracy: 0.5175 - val_loss: 0.7132 - val_accuracy: 0.4559 - lr: 5.0000e-04
Epoch 2/100
34/34 [==============================] - 3s 74ms/step - loss: 0.7208 - accuracy: 0.5506 - val_loss: 0.7340 - val_accuracy: 0.4485 - lr: 5.0000e-04
Epoch 3/100
34/34 [==============================] - 2s 71ms/step - loss: 0.7206 - accuracy: 0.5378 - val_loss: 0.6868 - val_accuracy: 0.5441 - lr: 5.0000e-04
Epoch 4/100
34/34 [==============================] - 2s 72ms/step - loss: 0.6909 - accuracy: 0.5267 - val_loss: 0.6846 - val_accuracy: 0.5809 - lr: 5.0000e-04
Epoch 5/100
34/34 [==============================] - 2s 73ms/step - loss: 0.7121 - accuracy: 0.5470 - val_loss: 0.6886 - val_accuracy: 0.5588 - lr: 5.0000e-04
Epoch 6/100
34/34 [==============================] - 2s 73ms/step - loss: 0.7124 - accuracy: 0.5157 - val_loss

In [7]:
# 输出所有文件的准确率
for label_name, accuracy_list in accuracies.items():
    for i, accuracy in enumerate(accuracy_list):
        print(f"File: {file_paths[i]}, Label: {label_name}, Accuracy: {accuracy:.4f}")

File: C:\Users\mjy\Desktop\Project_MSCM\INPUT_3.csv, Label: 未来7日涨幅, Accuracy: 0.5176
File: C:\Users\mjy\Desktop\Project_MSCM\INPUT_7.csv, Label: 未来7日涨幅, Accuracy: 0.6213
File: C:\Users\mjy\Desktop\Project_MSCM\INPUT_15.csv, Label: 未来7日涨幅, Accuracy: 0.6131
File: C:\Users\mjy\Desktop\Project_MSCM\INPUT_30.csv, Label: 未来7日涨幅, Accuracy: 0.5818
